In [2]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
import bitstring
from qiskit_aer import AerSimulator
from qiskit.transpiler.passes.synthesis import SolovayKitaev
from qiskit.synthesis import generate_basic_approximations
from qiskit.quantum_info import Operator

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.circuit.library import UnitaryGate

In [3]:
unit = np.array([[1,0],[0,1]])

t = np.array([[1,0],[0,np.exp((np.pi/4)*1j)]])

tdg = np.array([[1,0],[0,np.exp(-(np.pi/4)*1j)]])

h = np.array([[1,1],[1,-1]])

h = (1/np.sqrt(2))*h

test = np.array([[1,0],[0,np.exp((np.pi/8)*1j)]])

In [4]:
def dist(U,approx):
    ehm = np.conjugate(U.T)
    next = np.matmul(ehm, approx)
    hmm = np.abs(np.trace(next))
    return np.sqrt((2-hmm)/2)

def T(a,b):
    b = b*np.exp(1j*(np.pi/4))
    return a,b

def A(a,b):                             # = T adjoint
    b = b*np.exp(-1j*(np.pi/4))
    return a,b

def S(a,b):                             
    b = b*np.exp(1j*(np.pi/2))
    return a,b

def B(a,b):                             # = S adjoint
    b = b*np.exp(-1j*(np.pi/2))
    return a,b

def H(a,b):
    a = (1/(2**0.5))*(a+b)
    b = (1/(2**0.5))*(a-b)
    return a,b

def dustin(k: int, tgates: int):
    combs = ["A","T"]
    val_a , val_b = [],[]
    for i in range(tgates-1):
        all1 = [i+"A" for i in combs]
        all2 = [i+"T" for i in combs]
        combs = all1 + all2
    for i in range(len(combs)):
        seq = combs[i]
        a = 0
        b = 1
        for i in seq:
            if i == "A":
                a,b = H(a,b)
                a,b = A(a,b)
            if i == "T":
                a,b = H(a,b)
                a,b = T(a,b)
        val_a.append(a)
        val_b.append(b)
    angle = np.exp((1j*2*np.pi)/(2**(k+3)))
    winner = 0
    old = 3
    for i in range(len(val_a)):
        mine = mymatrix(combs[i])
        test = np.array([[1,0],[0,angle]])
        distance = dist(test, mine)
        if distance < old:
            winner = i
            old = distance
    return combs[winner]

def mymatrix(seq):
    ok1 = np.matmul(unit,unit)
    for i in range(len(seq)):
        if seq[i] == "A":
            ok1 = np.matmul(ok1,h)
            ok1 = np.matmul(ok1,tdg)
        if seq[i] == "T":
            ok1 = np.matmul(ok1,h)
            ok1 = np.matmul(ok1,t)
    return ok1

hmm= dustin(0, 8)

matrix = mymatrix(hmm)

print(hmm)
print(matrix)

AATTTAAT
[[ 0.97855339-0.1982233j -0.02144661+0.0517767j]
 [ 0.02144661+0.0517767j  0.97855339+0.1982233j]]
